In [31]:
# Get data from vastaav dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.model_selection import (
	TimeSeriesSplit, 
	GroupKFold,
	RandomizedSearchCV,
	GridSearchCV
)

from sklearn.ensemble import RandomForestRegressor

from sklearn.decomposition import PCA
from sklearn.preprocessing import (
	StandardScaler,
	MinMaxScaler, 
	OrdinalEncoder, 
	Binarizer,
	OneHotEncoder
)
from sklearn.metrics import (
	accuracy_score,
	r2_score,
	f1_score,
	root_mean_squared_error
)
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.compose import ColumnTransformer
import joblib, pickle
import xgboost as xgb
from utils import Database

sns.set_theme()

In [14]:
db = Database('../config.yaml')
engine = db.make_connection()

INFO 28-Oct 09:44 - Connected to the database.
INFO 28-Oct 09:44 - Connected to the database.


In [15]:
url = "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/refs/heads/master/data/2023-24/gws/merged_gw.csv"
new_url = "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/refs/heads/master/data/2024-25/gws/merged_gw.csv"

# Read the CSV file into a pandas DataFrame
# df = pd.read_csv(url)
df = db.get_data("""
	SELECT * FROM gameweek_stats
	WHERE season = '2023-24'
""")
df = df.drop('season', axis=1)

In [16]:
team_df = df.groupby(['team']).agg({
	'gw': 'max',
	'kickoff_time': 'max',
}).reset_index()

team_df = team_df.sort_values('team')
team_df['team_id'] = team_df.index + 1 # Create a unique id for the teams

## Create Features

In [17]:
df['kickoff_time'] = pd.to_datetime(df['kickoff_time'], utc=True)
# Sort by 'name' and 'kickoff_time' to ensure chronological order for each player
df = df.sort_values(by=["name", "kickoff_time"])


# Log returns on points
# df["log_return_points"] = np.log(df["total_points"] / df["total_points"].shift(1))


# Define a function to calculate the form (average score in last few games or last 30 days)
def calculate_player_form(player_df):
    # Take the most recent N games (e.g., last 5 games)
    last_game = player_df.kickoff_time.max()

    # Filter games that are within the last 30 days
    last_30_days_games = player_df[
        player_df["kickoff_time"] >= (last_game - pd.Timedelta(days=30))
    ]

    # Calculate average points in those last 30 days
    form = (
        last_30_days_games["total_points"].mean().round(1) if len(last_30_days_games) > 0 else 0
    )
    return form


# Group by player and calculate form based on their last 5 games (or you can change this to 30 days)
player_form = (
	df.groupby("name").apply(lambda x: calculate_player_form(x), include_groups=False).reset_index()
)

# Rename the columns for clarity
player_form.columns = ["name", "form"]

In [18]:
def calculate_form(group):
    window = "30D"
    return (
        group.set_index("kickoff_time")["total_points"]
        .rolling(window, min_periods=1)
        .mean()
		.round(1)
        .reset_index(name="form")
    )

# Form Query
form_query = """
WITH player_form AS (
    SELECT
        name,
        kickoff_time,
        --total_points,
        CAST(AVG(total_points) OVER (
            PARTITION BY name
            ORDER BY kickoff_time
            RANGE BETWEEN INTERVAL '30 days' PRECEDING AND CURRENT ROW
        ) AS DECIMAL(3, 1)) AS form
    FROM gameweek_stats
)
SELECT pf.* FROM player_form pf
JOIN gameweek_stats gs
ON pf.name = gs.name
AND pf.kickoff_time = gs.kickoff_time
WHERE gs.season = :season;"""

form_df = db.get_data(form_query, params={'season':'2023-24'})
form_df['kickoff_time'] = pd.to_datetime(form_df['kickoff_time'], utc=True)
# print(form_df.kickoff_time.)
# df["form"] = 
# res = form_df[["name", "kickoff_time", "form"]].merge(
#     df, on=["name", "kickoff_time"], how="inner"
# )

df = pd.merge(form_df, df, on=['name', 'kickoff_time'], how='inner')
# form_df = db.get_data(form_query, params=['2023-24'])
# df[df.columns[:5]].head()


### Point difference

In [41]:
# Create a column that shows the difference in points between the current game and the previous game
df["points_diff"] = df["total_points"].diff()
pts_diff = df[['name', 'kickoff_time', 'total_points', 'points_diff']]

chippy_chips = df.query('name == "Erling Haaland"').set_index('kickoff_time')[['total_points', 'form', 'points_diff']]
px.line(chippy_chips[['total_points', 'form']], title='Cole Palmer Points and Form')

In [24]:
res = df.groupby("name")["minutes"].apply(
	lambda x: (x > 60).astype(int).cumsum()).reset_index(name="streak")

res

,name,level_1,streak
0,Aaron Connolly,0,0
1,Aaron Connolly,1,0
2,Aaron Connolly,2,0
3,Aaron Connolly,3,0
4,Aaron Connolly,4,0
...,...,...,...
29720,Đorđe Petrović,29720,14
29721,Đorđe Petrović,29721,15
29722,Đorđe Petrović,29722,16
29723,Đorđe Petrović,29723,17


In [104]:
form_df = df.groupby("name").apply(calculate_form).reset_index(level=0)
# df = df.merge(form_df[['name', 'kickoff_time', 'form']], on=['name', 'kickoff_time'], how='left')

/var/folders/0g/b4_r77h96ss9xc40r9fk2ld40000gn/T/ipykernel_90948/1373165700.py:1: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [89]:
# Create an exponentially weighted moving average (EWMA) of the form
player_form["ewma_form"] = player_form["form"].ewm(span=5).mean().round(2)

In [90]:
df['kickoff_time'] = pd.to_datetime(df['kickoff_time'])
df.sort_values(by=['kickoff_time'], inplace=True)
# Create a 'points_per_match' feature
# Group by player name and calculate the average points per match over the season
df_grouped = (
    df.groupby("name").agg({"total_points": "sum", "minutes": "count"}).reset_index()
)

# Calculate points per match over multiple GWs
df_grouped["points_per_match"] = df_grouped["total_points"] / df_grouped["minutes"]
df_grouped = df_grouped.round(2)
# df = df.merge(df_grouped[["name", "points_per_match"]], on="name")
# df = pd.concat([df, df_grouped], axis=1)


# Create a 'points_per_mil' feature
df_grouped["value"] = df["value"]
df_grouped["points_per_mil"] = df_grouped["points_per_match"] / df["value"]


In [18]:
# Double gw mask
dgw_mask = df.groupby(['name', 'gw']).size().reset_index(name='games_in_gw')


## Preprocessing

In [91]:
ord_enc = OrdinalEncoder()
one_hot = OneHotEncoder()
scaler = StandardScaler()
mm_scaler = MinMaxScaler()
bin_enc = Binarizer()

#### Hyperparameters

In [92]:
parameters = {
	'n_estimators': [100, 200, 300, 400, 500],
	# 'max_depth': [3, 5, 7, 9, 11, 13, 15],
	'lambda': [0.1, 0.5, 1, 1.5, 2],
	'learning_rate': [0.01, 0.05, 0.1, 0.15, 0.2],
	'gamma': [0, 0.1, 0.2, 0.3, 0.4, 0.5],
}

## Forwards

In [93]:
forwards_df = df[df['position'] == 'FWD'].copy()

forwards_df.loc[:, 'points_per_mil'] = forwards_df['total_points'] / forwards_df['value']

## Midfielders

In [94]:
midfield_df = df.loc[df['position'] == 'MID'].copy()

midfield_df.loc[:, 'points_per_mil'] = midfield_df['total_points'] / midfield_df['value']
midfield_df = midfield_df.merge(team_df[['team', 'team_id']], 'left', on='team')

# midfield_df["assists_rolling_5"] = df.groupby("name")["assists"].transform(
#     lambda x: x.rolling(5, min_periods=1).mean()
# )

# Interaction features
midfield_df["expected_assists_minutes"] = df["expected_assists"] * df["minutes"]


In [95]:
midfield_df.drop(['saves', 'expected_goals_conceded'], axis=1, inplace=True)

In [96]:
numeric_cols = midfield_df.select_dtypes(exclude=['object', 'datetime64[ns, UTC]', 'bool'])
numeric_cols.drop(columns=['round'], inplace=True)

midfield_X = midfield_df.drop(columns=['total_points', 'name', 'team', 'position', 'kickoff_time'])
midfield_y = midfield_df['total_points']
print(midfield_X.columns)

Index(['xp', 'assists', 'bonus', 'bps', 'clean_sheets', 'creativity',
       'element', 'expected_assists', 'expected_goal_involvements',
       'expected_goals', 'fixture', 'goals_conceded', 'goals_scored',
       'ict_index', 'influence', 'minutes', 'opponent_team', 'own_goals',
       'penalties_missed', 'penalties_saved', 'red_cards', 'round', 'selected',
       'starts', 'team_a_score', 'team_h_score', 'threat', 'transfers_balance',
       'transfers_in', 'transfers_out', 'value', 'was_home', 'yellow_cards',
       'gw', 'form', 'points_per_mil', 'team_id', 'expected_assists_minutes'],
      dtype='object')


In [97]:
# midfield_df.plot(x='total_points', y='points_per_mil', kind='scatter')
px.scatter(midfield_df, x='value', y='points_per_mil', color='team')

# px.box(midfield_df, x='team', y='points_per_mil', color='team')

### Train on midfielders

In [98]:

# Get the list of columns that are numeric using np
numeric_cols = df.select_dtypes(include=[np.number]).columns
numeric_cols.drop(
    ["total_points", "expected_goals_conceded", "saves"]
)

Index(['xp', 'assists', 'bonus', 'bps', 'clean_sheets', 'creativity',
       'element', 'expected_assists', 'expected_goal_involvements',
       'expected_goals', 'fixture', 'goals_conceded', 'goals_scored',
       'ict_index', 'influence', 'minutes', 'opponent_team', 'own_goals',
       'penalties_missed', 'penalties_saved', 'red_cards', 'round', 'selected',
       'starts', 'team_a_score', 'team_h_score', 'threat', 'transfers_balance',
       'transfers_in', 'transfers_out', 'value', 'yellow_cards', 'gw', 'form'],
      dtype='object')

In [99]:
numeric_cols = list(df.select_dtypes(include=[np.number]).columns
.drop(['total_points', 'expected_goals_conceded', 'saves']))

preprocessor = ColumnTransformer(
    transformers=[
        (
            "scaler",
            scaler,
			numeric_cols
        ),
        ("mm_scaler", mm_scaler, ["value"]),
        ("bin_enc", bin_enc, ["was_home"]),
		# ("one_hot", ord_enc, ['team', 'position']),
    ],
	remainder='drop'
)

# Initialize and fit the XGBoost model
xgb_model = xgb.XGBRegressor(n_estimators=500, learning_rate=0.1, random_state=42)

# Create a pipeline
pipe = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        # ("pca", PCA(n_components=0.95)),
        ("model", xgb_model),
    ]
)

In [101]:
# Using TimeSeriesSplit to create a time-based cross-validation strategy
tscv = TimeSeriesSplit(n_splits=5)  # 5 splits

splits = list(tscv.split(midfield_X))

print(f'{midfield_X.columns=}')

# Loop through the time series splits
for train_index, test_index in tscv.split(midfield_X):
	# Use indices from TimeSeriesSplit to extract train/test features
	X_train, X_test = midfield_X.iloc[train_index, :], midfield_X.iloc[test_index, :]

	# Use the same indices to extract corresponding target variable (total_points)
	y_train, y_test = midfield_y.iloc[train_index], midfield_y.iloc[test_index]

	# xgb_model.fit(X_train, y_train)
	pipe.fit(X_train, y_train)
	y_pred = pipe.predict(X_test)


r2 = r2_score(y_test, y_pred)
# r2_hgb = r2_score(y_test)
print(f"R2 Score: {r2}")

midfield_X.columns=Index(['xp', 'assists', 'bonus', 'bps', 'clean_sheets', 'creativity',
       'element', 'expected_assists', 'expected_goal_involvements',
       'expected_goals', 'fixture', 'goals_conceded', 'goals_scored',
       'ict_index', 'influence', 'minutes', 'opponent_team', 'own_goals',
       'penalties_missed', 'penalties_saved', 'red_cards', 'round', 'selected',
       'starts', 'team_a_score', 'team_h_score', 'threat', 'transfers_balance',
       'transfers_in', 'transfers_out', 'value', 'was_home', 'yellow_cards',
       'gw', 'form', 'points_per_mil', 'team_id', 'expected_assists_minutes'],
      dtype='object')
R2 Score: 0.9932615160942078


In [69]:
# Feature importance for Random Forest
importances = pipe.named_steps['model'].feature_importances_
features = midfield_X.columns

# Sort by importance
indices = importances.argsort()[::-1]

# Plot
px.bar(y=importances[indices], x=features[indices], title="Feature Importance")

### Hyperparameter Tuning

In [70]:
random_search_xgb = RandomizedSearchCV(
	xgb_model, 
	parameters, 
	n_iter=10, 
	cv=tscv, 
	verbose=2, 
	random_state=42, 
	n_jobs=-1,
	scoring='neg_mean_squared_error'
)

random_search_xgb.fit(midfield_X, midfield_y)

# Get the best parameters
best_params = random_search_xgb.best_params_
print(f'{best_params=}')

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[CV] END gamma=0.4, lambda=0.1, learning_rate=0.05, n_estimators=200; total time=   0.1s
[CV] END gamma=0.2, lambda=2, learning_rate=0.05, n_estimators=300; total time=   0.2s
[CV] END gamma=0.4, lambda=0.1, learning_rate=0.05, n_estimators=200; total time=   0.2s
[CV] END gamma=0.4, lambda=0.1, learning_rate=0.05, n_estimators=200; total time=   0.2s
[CV] END gamma=0.2, lambda=2, learning_rate=0.05, n_estimators=300; total time=   0.2s
[CV] END gamma=0.4, lambda=0.1, learning_rate=0.05, n_estimators=200; total time=   0.3s
[CV] END gamma=0.2, lambda=2, learning_rate=0.05, n_estimators=300; total time=   0.3s
[CV] END gamma=0.4, lambda=0.1, learning_rate=0.05, n_estimators=200; total time=   0.3s
[CV] END gamma=0.2, lambda=2, learning_rate=0.05, n_estimators=300; total time=   0.4s
[CV] END gamma=0.1, lambda=0.1, learning_rate=0.05, n_estimators=400; total time=   0.2s
[CV] END gamma=0.2, lambda=2, learning_rate=0.05, n_estimators=300; total time=   0.4s
[CV] END gamma=0.1, lambda=0.1,

### Evaluate on test sets

In [71]:
r2 = r2_score(y_test, y_pred)
print(f"Random Forest R²: {r2}")

import datetime
# Save the model
# joblib.dump(xgb_model, f'../models/xgb_model_{datetime.datetime.now()}.pkl')
day = datetime.datetime.now().strftime('%Y%m%d')
# joblib.dump(pipe, f'../models/xgb_model_{day}.joblib')

Random Forest R²: 0.9935234189033508


### Load and use model

In [ ]:
model = joblib.load('../models/xgb_model_20241024.joblib')
# model = pickle.load(open('../models/xgb_model.pkl', 'rb'))
# Predict on data from the 2024-25 season
new_url = "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/refs/heads/master/data/2024-25/gws/merged_gw.csv"
new_df = pd.read_csv(new_url)


new_df = new_df.loc[new_df['position'] == 'MID'].copy()
new_df.loc[:, 'points_per_mil'] = new_df['total_points'] / new_df['value']
new_df = new_df.merge(team_df[['team', 'team_id']], 'left', on='team')
new_df['kickoff_time'] = pd.to_datetime(new_df['kickoff_time'])

# Interaction features
new_df["expected_assists_minutes"] = new_df["expected_assists"] * new_df["minutes"]

teams = new_df['team'].unique().tolist()

new_df.drop(['saves', 'expected_goals_conceded'], axis=1, inplace=True)

# new_df['assists_rolling_5'] = new_df.groupby('name')['assists'].transform(
# 	lambda x: x.rolling(5, min_periods=1).mean()
# )

new_df_form = new_df.groupby("name").apply(lambda x: calculate_player_form(x), include_groups=False).reset_index()
# preprocessor.fit(new_df)
new_X = new_df.drop(columns=['total_points', 'name', 'team', 'position', 'kickoff_time'])
new_X.columns = new_X.columns.str.lower()
og_x_cols = sorted(list(pipe.feature_names_in_))
new_x_cols = sorted(list(new_X.columns))

# print(f'{new_x_cols - og_x_cols}')
new_X = preprocessor.fit_transform(new_X)
new_y = new_df['total_points']


print(f'input feature shape: \n{og_x_cols}\n{new_x_cols}')
print(len(og_x_cols), len(new_x_cols))
# Predict on the new data
new_pred = model.predict(new_X)

In [18]:
# Calculate the R² score
r2 = r2_score(new_y, new_pred)
print(f"R² Score on new data: {r2}")

NameError: name 'new_pred' is not defined

## Defenders

In [256]:
defenders_df = df.loc[df['position'] == 'DEF'].copy()

defenders_df['points_per_mil'] = defenders_df['total_points'] / defenders_df['value']